In [ ]:
import numpy as np
from numba import njit
import matplotlib.pyplot as plt

# ─────────────────────────────────────
# 1. Global constants & Pauli operators
# ─────────────────────────────────────
PI        = np.pi
B0     = 1.0
omega0 = 1.0
h_FD      = 1.0e-4
total_T   = 2.0
N_slices  = 500
tau       = total_T / N_slices
theta_rot = PI / 3
n_epochs  = 1000
J         = 0.01

# Pauli matrices
σx = np.array([[0,1],[1,0]],complex)
σy = np.array([[0,-1j],[1j,0]],complex)
σz = np.array([[1,0],[0,-1]],complex)
I2 = np.eye(2, dtype=complex)

σ1x,σ1y,σ1z = np.kron(σx,I2),np.kron(σy,I2),np.kron(σz,I2)
ZZ          = np.kron(σz,σz)

# Bell-state projectors
bell = np.array([[1,0,0,-1],
                 [0,1,1,0],
                 [0,1,-1,0],
                 [1,0,0,1]],complex)/np.sqrt(2)
projs = np.array([np.outer(v,v.conj()) for v in bell])

ψ0  = np.array([1,0,0,1],complex)/np.sqrt(2)
ρ0  = np.outer(ψ0,ψ0.conj())

# ─────────────────────────────────────
# 2. Unitary block
# ─────────────────────────────────────
@njit(inline='always')
def U4_full(vx, vy, vz, Jcouple, dt):
    hz = np.array([vz+Jcouple, vz-Jcouple])
    U  = np.zeros((4,4),dtype=np.complex128)
    for blk in range(2):
        mag = np.sqrt(vx*vx + vy*vy + hz[blk]*hz[blk])
        if mag < 1e-14:
            cs=1.0; ss=0.0
        else:
            cs = np.cos(mag*dt)
            ss = np.sin(mag*dt)/mag
        a = cs-1j*ss*hz[blk]; d = cs+1j*ss*hz[blk]
        b = -1j*ss*(vx-1j*vy); c = -1j*ss*(vx+1j*vy)
        if blk==0:
            U[0,0]=a; U[0,2]=b; U[2,0]=c; U[2,2]=d
        else:
            U[1,1]=a; U[1,3]=b; U[3,1]=c; U[3,3]=d
    return U

@njit
def probabilities(theta_rot, tau, n_slices,
                  B, ω,
                  Vx, Vy, Vz, Jcouple,
                  projs):
    ρ = ρ0.copy()
    for t in range(n_slices):
        t_now = t*tau
        # --- target fields ---
        vx_t = -B * np.cos(ω*t_now)
        vy_t = -B * np.sin(ω*t_now)
        vz_t = 0.0
        # --- control fields ---
        vx_t += Vx[t]; vy_t += Vy[t]; vz_t += Vz[t]
        U  = U4_full(vx_t, vy_t, vz_t, Jcouple, tau)
        ρ  = U @ ρ @ U.conj().T

    # measurement-axis rotation
    c = 1/np.sqrt(3)
    Urot = U4_full(c,c,c,0.0,theta_rot)
    ρ = Urot @ ρ @ Urot.conj().T

    p = np.empty(4)
    for k in range(4):
        p[k] = np.real(np.trace(ρ @ projs[k]))
    return p

@njit
def f0_and_trace(Vx,Vy,Vz,
                 theta_rot,tau,n_slice,
                 B0,ω0,h,Jcouple,projs):
    # base + ±h on B and ω
    P0   = probabilities(theta_rot,tau,n_slice,B0,   ω0,   Vx,Vy,Vz,Jcouple,projs)
    PBp  = probabilities(theta_rot,tau,n_slice,B0+h, ω0,   Vx,Vy,Vz,Jcouple,projs)
    PBm  = probabilities(theta_rot,tau,n_slice,B0-h, ω0,   Vx,Vy,Vz,Jcouple,projs)
    Pwp  = probabilities(theta_rot,tau,n_slice,B0,   ω0+h,Vx,Vy,Vz,Jcouple,projs)
    Pwm  = probabilities(theta_rot,tau,n_slice,B0,   ω0-h,Vx,Vy,Vz,Jcouple,projs)

    grad = np.empty((2,4))
    grad[0]=(PBp-PBm)/(2*h)
    grad[1]=(Pwp-Pwm)/(2*h)

    F=np.zeros((2,2))
    for i in range(2):
        for j in range(2):
            acc=0.0
            for k in range(4):
                acc += grad[i,k]*grad[j,k]/max(P0[k],1e-12)
            F[i,j]=acc
    Cov = np.linalg.inv(F)
    tr_inv = Cov[0,0]+Cov[1,1]
    f0 = 1.0/(1/F[0,0]+1/F[1,1])
    return f0,tr_inv

# ─────────────────────────────────────
# 3. Initial control: Optimal control for H0 without the Ising term
# ─────────────────────────────────────
B_ctrl   = B0
ω_ctrl   = omega0

V1x=np.zeros(N_slices); V1y=np.zeros(N_slices); V1z=np.zeros(N_slices)
for t in range(N_slices):
    t_now=t*tau
    V1x[t] =  B_ctrl*np.cos(ω_ctrl*t_now)
    V1y[t] =  B_ctrl*np.sin(ω_ctrl*t_now)
    V1z[t] =  ω_ctrl/2

# ─────────────────────────────────────
# 3. Initial control: Control obtained by GRAPE in the former case
# ─────────────────────────────────────
# V1x=np.zeros(N_slices); V1y=np.zeros(N_slices); V1z=np.zeros(N_slices)
# V1x = [0.6800901711, -0.4374226975, -1.4919140717, -2.4810770645, -3.3739998061, -4.1676732722, -4.8248597591, -5.3369975502, -5.6814745432, -5.8375214980, -5.8032334515, -5.5686984400, -5.1242488184, -4.4840134943, -3.6510818732, -2.6433266226, -1.4781976731, -0.1713559462, 1.2401158793, 2.7370421302, 4.2876569827, 5.8703841076, 7.4566098423, 9.0055313277, 10.5204301464, 11.9435194457, 13.2846693674, 14.5177378578, 15.6254136834, 16.6040512128, 17.4481676375, 18.1589419764, 18.7315349488, 19.1840205993, 19.5108228983, 19.7243761482, 19.8267748702, 19.8415797480, 19.7536043304, 19.5718734822, 19.3147653282, 18.9542341019, 18.5115906919, 17.9692268107, 17.3274583275, 16.5898663257, 15.7538350706, 14.8155620917, 13.7813559955, 12.6584471746, 11.4441024857, 10.1624354851, 8.8259620299, 7.4393082537, 6.0291574492, 4.6112050965, 3.1998385536, 1.8121706736, 0.4792164952, -0.7815879653, -1.9643924331, -3.0429334422, -4.0049841148, -4.8373458832, -5.5197321479, -6.0612487240, -6.4457555810, -6.6811447325, -6.7624678007, -6.7120386786, -6.5279324779, -6.2267359170, -5.8260727312, -5.3388250920, -4.7878162272, -4.1895858555, -3.5616477933, -2.9096697235, -2.2568193825, -1.6175988128, -0.9869616417, -0.3800239208, 0.1965738368, 0.7503471801, 1.2679371984, 1.7474604972, 2.1840800582, 2.5578876672, 2.8812161220, 3.1245813586, 3.2972945353, 3.3785904483, 3.3755291791, 3.2946918406, 3.1459347243, 2.9441681536, 2.7163349354, 2.4813329830, 2.2526851005, 2.0426682210, 1.8632578468, 1.7086803827, 1.5830899789, 1.4796740769, 1.3861625551, 1.3069702755, 1.2406781417, 1.1788069872, 1.1287749492, 1.0898410279, 1.0607072906, 1.0460278884, 1.0395380132, 1.0442763371, 1.0481401920, 1.0614906808, 1.0731846690, 1.0837630878, 1.0868087053, 1.0849565335, 1.0744445156, 1.0474740063, 1.0169072822, 0.9778293691, 0.9288438011, 0.8678163349, 0.8084523280, 0.7425707440, 0.6703878023, 0.6052319878, 0.5335166832, 0.4678971281, 0.4087600510, 0.3545256843, 0.3034519939, 0.2618831715, 0.2314249560, 0.2098699490, 0.2014791343, 0.2000046570, 0.2106329601, 0.2254826743, 0.2514309796, 0.2869304554, 0.3263311517, 0.3720016522, 0.4261595850, 0.4861740980, 0.5388829257, 0.5943899517, 0.6526327726, 0.7030882011, 0.7550689092, 0.8048376102, 0.8501456725, 0.8798300259, 0.9148133991, 0.9413582719, 0.9597838624, 0.9705479990, 0.9696801482, 0.9680654572, 0.9565290041, 0.9329054082, 0.9025546750, 0.8667602103, 0.8226688380, 0.7792925388, 0.7361118762, 0.6998992497, 0.6690491936, 0.6503838936, 0.6364995537, 0.6366857996, 0.6468319989, 0.6671872511, 0.6835536868, 0.7045016854, 0.7194487494, 0.7354671809, 0.7488434679, 0.7515554058, 0.7544819480, 0.7486392443, 0.7446591159, 0.7356904521, 0.7208134559, 0.7093142790, 0.7006019392, 0.6944935284, 0.6943923737, 0.6948534052, 0.6958242786, 0.7001438089, 0.7078780621, 0.7107392055, 0.7200990734, 0.7222639692, 0.7251338360, 0.7274386707, 0.7309397514, 0.7296336405, 0.7332407687, 0.7353707681, 0.7326093514, 0.7312987848, 0.7272697645, 0.7284223312, 0.7259570776, 0.7249686696, 0.7156530828, 0.7119606169, 0.6992952627, 0.6871830308, 0.6774622064, 0.6636579851, 0.6445691152, 0.6255549922, 0.6096519602, 0.5889457004, 0.5728679611, 0.5509594901, 0.5315358909, 0.5126881425, 0.5001792665, 0.4912298489, 0.4845498465, 0.4795516274, 0.4802101177, 0.4824929639, 0.4956700807, 0.5161124419, 0.5399912845, 0.5626260669, 0.5948919051, 0.6280524596, 0.6618330467, 0.6934726244, 0.7203410707, 0.7483009771, 0.7645461976, 0.7724156532, 0.7709131868, 0.7648045421, 0.7440469361, 0.7141352849, 0.6813010405, 0.6507662814, 0.6265630541, 0.6018040986, 0.5890996742, 0.5753929539, 0.5675881608, 0.5554992830, 0.5390939423, 0.5203649548, 0.4973158020, 0.4661582617, 0.4428666522, 0.4197455940, 0.4173889340, 0.4156807375, 0.4303056320, 0.4513800929, 0.4735010196, 0.4974777449, 0.5168508725, 0.5317557945, 0.5441556742, 0.5411321461, 0.5387114557, 0.5269886533, 0.5087054529, 0.4909995984, 0.4632279244, 0.4364717029, 0.4104804269, 0.3774652229, 0.3520983989, 0.3241746552, 0.2918232854, 0.2713251712, 0.2529855613, 0.2338426488, 0.2266935133, 0.2175580205, 0.2182009073, 0.2273093491, 0.2397333520, 0.2574556506, 0.2821984569, 0.3087643895, 0.3456377089, 0.3761520777, 0.4123894021, 0.4524948395, 0.4835491492, 0.5147591539, 0.5441328134, 0.5615129876, 0.5758150772, 0.5798108367, 0.5831143476, 0.5756210369, 0.5619409548, 0.5394502606, 0.5144592510, 0.4880893603, 0.4566813066, 0.4205795711, 0.3877878340, 0.3482277663, 0.3160749009, 0.2813739224, 0.2529227587, 0.2180530679, 0.1925091468, 0.1667477592, 0.1462186505, 0.1281805988, 0.1216145320, 0.1218424588, 0.1441330163, 0.1755928168, 0.2239674860, 0.2957972404, 0.3757172813, 0.4677636639, 0.5593104016, 0.6362847046, 0.6953563663, 0.7194856261, 0.7139040732, 0.6670565047, 0.5859672911, 0.4806451710, 0.3551969649, 0.2280408828, 0.1141823095, 0.0275121611, -0.0158665481, -0.0227655645, 0.0247748193, 0.1015999711, 0.2140488746, 0.3461006295, 0.4825467717, 0.6220479518, 0.7559878985, 0.8902717199, 1.0080937709, 1.1167078947, 1.2063352205, 1.2732517061, 1.3153096465, 1.3273264030, 1.2988891539, 1.2217102953, 1.1057252784, 0.9414793760, 0.7349996450, 0.4932335073, 0.2170989993, -0.0789401129, -0.3831190744, -0.6850091892, -0.9741164080, -1.2391164120, -1.4630590952, -1.6479826705, -1.7738883509, -1.8417124487, -1.8436214467, -1.7864606638, -1.6713365371, -1.4910688732, -1.2654695905, -0.9943768310, -0.6938864206, -0.3733284203, -0.0398970240, 0.2933926231, 0.6098000549, 0.9020491277, 1.1617987144, 1.3784211069, 1.5466433782, 1.6686071892, 1.7353124449, 1.7467138647, 1.7158450017, 1.6332919830, 1.5103046333, 1.3504362712, 1.1608307650, 0.9475549328, 0.7203364794, 0.4802217797, 0.2348858381, -0.0076837850, -0.2423178185, -0.4613754208, -0.6565248338, -0.8267344747, -0.9553994556, -1.0398934967, -1.0763290827, -1.0636284884, -1.0024040559, -0.8997878399, -0.7723761659, -0.6320641670, -0.4852440164, -0.3396249309, -0.1968111651, -0.0492134417, 0.0938803831, 0.2240555750, 0.3170481376, 0.3330167908, 0.2339418797, -0.0120359811, -0.4162117351, -0.9659299140, -1.6395552158, -2.3872146647, -3.1763944116, -3.9623265921, -4.7212735030, -5.4239163513, -6.0536675571, -6.5993120416, -7.0410057239, -7.3677105402, -7.5613848543, -7.5982000295, -7.4587701334, -7.1248144761, -6.5724023132, -5.7919992822, -4.7724828251, -3.5139636734, -2.0253733556, -0.3128021083, 1.5960627555, 3.6774713982, 5.9003915851, 8.2108320024, 10.5596648803, 12.8785523765, 15.0997980100, 17.1408052733, 18.9157325082, 20.3396101739, 21.3298422689, 21.8214501239, 21.7603217117, 21.1198717388, 19.9014326091, 18.1273053567, 15.8509395111, 13.1446261770, 10.0888114028, 6.7831690068, 3.3210348906, -0.1949552090, -3.6663683143, -6.9984013627, -10.0927895539, -12.8623518975, -15.2265873529, -17.1206670380, -18.4966659988, -19.3298998359, -19.6143233586, -19.3801752458, -18.6715595675, -17.5531607785, -16.1046604362, -14.4078044006, -12.5512020214, -10.6136259713, -8.6770545853, -6.8101985965, -5.0801274678, -3.5481676219, -2.2686446637, -1.2980422636, -0.6870395457, -0.4858591307, -0.7467542345, -1.5242554678, -2.8744406809, -4.8625804935, -7.5598995730, -11.0468175143, -15.4024060137, -20.6997451198, -26.9677031466, -34.1392324656]
# V1y = [6.2531789903, 5.6718614901, 5.0463875745, 4.3859138186, 3.7408040851, 3.1172177360, 2.5370896685, 2.0022187145, 1.5324118022, 1.1207556459, 0.7781755625, 0.4932456516, 0.2644819369, 0.0967920922, -0.0291973352, -0.0926805025, -0.1153619025, -0.1017284328, -0.0420808363, 0.0512243167, 0.1824016284, 0.3454451353, 0.5334120488, 0.7463964124, 0.9754906916, 1.2208540945, 1.4679593724, 1.7190739312, 1.9642130080, 2.1986893116, 2.4224463760, 2.6220925948, 2.8056279213, 2.9694474910, 3.1044648404, 3.2128226104, 3.3056345248, 3.3666471729, 3.4111147162, 3.4255730333, 3.4215548591, 3.3854568293, 3.3360883704, 3.2636275047, 3.1658528662, 3.0493538158, 2.9148550905, 2.7563963436, 2.5861893377, 2.3961387529, 2.1961697753, 1.9806962510, 1.7607680661, 1.5248768211, 1.2907202390, 1.0451205186, 0.7981985113, 0.5411674621, 0.2861196842, 0.0285671624, -0.2272511800, -0.4806418716, -0.7296507579, -0.9656519083, -1.1913410435, -1.4081365222, -1.5954401946, -1.7562583756, -1.8853235351, -1.9812812307, -2.0340400183, -2.0467517747, -2.0018253386, -1.9174234567, -1.7740335044, -1.5796476708, -1.3358808230, -1.0513689556, -0.7373697087, -0.3966905616, -0.0435768022, 0.3065329323, 0.6384301917, 0.9248093914, 1.1569525206, 1.3157380457, 1.3969982586, 1.3871482629, 1.2871981683, 1.1068654876, 0.8597117078, 0.5613882286, 0.2481185712, -0.0635954311, -0.3521225166, -0.5859518665, -0.7625912200, -0.8667690204, -0.9210559518, -0.9220291475, -0.8814294799, -0.8249020498, -0.7453799206, -0.6587593764, -0.5685353732, -0.4784007634, -0.3852057455, -0.2851383788, -0.1979944943, -0.0936829839, -0.0024545595, 0.0921183734, 0.1751371109, 0.2539184021, 0.3221437928, 0.3762676022, 0.4251600436, 0.4662679219, 0.4937410539, 0.5142633606, 0.5265693002, 0.5237851697, 0.5220746313, 0.5150447373, 0.4985373961, 0.4756486308, 0.4502709128, 0.4248177587, 0.3985685836, 0.3673188432, 0.3333642473, 0.3051215523, 0.2794730334, 0.2496482764, 0.2356501499, 0.2179514916, 0.2099001015, 0.2115788395, 0.2096427435, 0.2258101143, 0.2423566418, 0.2628526809, 0.2884487665, 0.3201576975, 0.3528787774, 0.3924647841, 0.4227581409, 0.4591706304, 0.4903868044, 0.5123158088, 0.5368269465, 0.5514336147, 0.5636148015, 0.5736812699, 0.5829514540, 0.5823362541, 0.5823709743, 0.5828027309, 0.5856868967, 0.5997815008, 0.6060123236, 0.6148185293, 0.6269096010, 0.6370099019, 0.6523968689, 0.6631761130, 0.6783119969, 0.6795509154, 0.6765592763, 0.6662793431, 0.6557334513, 0.6491833733, 0.6355967348, 0.6343193024, 0.6301777915, 0.6427532821, 0.6590133359, 0.6839365480, 0.7095295907, 0.7370440866, 0.7563221791, 0.7723978038, 0.7822971634, 0.7873647587, 0.7769809886, 0.7685732142, 0.7508696306, 0.7265689684, 0.7022566478, 0.6840102228, 0.6583596661, 0.6420204260, 0.6327489049, 0.6213365789, 0.6185405266, 0.6234333728, 0.6320840557, 0.6428790949, 0.6558984581, 0.6703070801, 0.6859656763, 0.7028943266, 0.7119596939, 0.7271410632, 0.7394275950, 0.7566108959, 0.7608037760, 0.7692185341, 0.7742334242, 0.7740193046, 0.7810761278, 0.7809290136, 0.7821157932, 0.7867287267, 0.7868424854, 0.7822940576, 0.7829997759, 0.7811412229, 0.7855383203, 0.7864383745, 0.7901304709, 0.7822792871, 0.7864895555, 0.7892531671, 0.7901915452, 0.7880749703, 0.7849192349, 0.7771275255, 0.7714558697, 0.7668721804, 0.7593391278, 0.7504241053, 0.7396671796, 0.7300436630, 0.7252420030, 0.7266523342, 0.7292470301, 0.7357737811, 0.7529248522, 0.7731120179, 0.7976347920, 0.8250722550, 0.8494935277, 0.8790632541, 0.8932568246, 0.8999371783, 0.8962547870, 0.8839528286, 0.8545187855, 0.8190275182, 0.7897274825, 0.7505162081, 0.7220969632, 0.7006392462, 0.6854198891, 0.6780672762, 0.6758347110, 0.6795176674, 0.6858560305, 0.6895116600, 0.6973271462, 0.7106511907, 0.7280059988, 0.7535665549, 0.7781905021, 0.8095120312, 0.8428083332, 0.8786593180, 0.9085255443, 0.9356510663, 0.9577047635, 0.9737881803, 0.9791710448, 0.9792824075, 0.9727103473, 0.9615021391, 0.9456451060, 0.9301201064, 0.9115353325, 0.8946635679, 0.8782874291, 0.8665799840, 0.8526067103, 0.8462257001, 0.8409474415, 0.8381165914, 0.8389880228, 0.8456736962, 0.8593869616, 0.8722972215, 0.8909208184, 0.9095436198, 0.9302525762, 0.9531848365, 0.9754598856, 0.9986900925, 1.0230807807, 1.0456605585, 1.0679597640, 1.0834489363, 1.0998675117, 1.1153125435, 1.1225167033, 1.1193472932, 1.1187624024, 1.1128936581, 1.0957541914, 1.0745836910, 1.0500081677, 1.0186915837, 0.9835846923, 0.9443013240, 0.9055140085, 0.8646993571, 0.8221382903, 0.7858848975, 0.7555420714, 0.7346355784, 0.7209357970, 0.7181409710, 0.7285118631, 0.7484457556, 0.7806606251, 0.8151631537, 0.8560177502, 0.8891468458, 0.9197105126, 0.9349559061, 0.9299800488, 0.9140281361, 0.8889186202, 0.8524577919, 0.8155660714, 0.7844983564, 0.7709775089, 0.7724434778, 0.7933344136, 0.8321419095, 0.8877899047, 0.9523344781, 1.0189626826, 1.0724984626, 1.1084576395, 1.1116788645, 1.0882098103, 1.0323645446, 0.9524847950, 0.8587943828, 0.7639647174, 0.6724732698, 0.6001480127, 0.5516436515, 0.5275955414, 0.5402984487, 0.5805928684, 0.6474601911, 0.7329534525, 0.8332704287, 0.9434762321, 1.0464500941, 1.1487704145, 1.2352946411, 1.3024601057, 1.3499972911, 1.3753406483, 1.3845104062, 1.3702120438, 1.3407654103, 1.2959185788, 1.2450609141, 1.1832808259, 1.1238778407, 1.0608375858, 1.0008072151, 0.9431938409, 0.8965216994, 0.8554759408, 0.8222880139, 0.7949964417, 0.7846924407, 0.7721021549, 0.7676367434, 0.7715704763, 0.7784321926, 0.7855012303, 0.7963120515, 0.8091655915, 0.8215426785, 0.8312461910, 0.8468264546, 0.8642684905, 0.8892650729, 0.9171559532, 0.9544513673, 1.0043429701, 1.0629664072, 1.1277970345, 1.2001285217, 1.2758615187, 1.3451803753, 1.3987832937, 1.4384925478, 1.4501644643, 1.4325461226, 1.3854208052, 1.3065084531, 1.2089950688, 1.1007250404, 0.9910551059, 0.8928172517, 0.8112343469, 0.7421631699, 0.6911823498, 0.6483682376, 0.6076506336, 0.5823506994, 0.5822237547, 0.6303677047, 0.7409926009, 0.9221853217, 1.1535805813, 1.3982051456, 1.6087314149, 1.7387450106, 1.7461306843, 1.6228284765, 1.3558741533, 0.9853038257, 0.5346609985, 0.0495116235, -0.4240724915, -0.8543497419, -1.2067987134, -1.4668463494, -1.6197938496, -1.6723192055, -1.6231818084, -1.4930191107, -1.2831394392, -1.0200495526, -0.7069742273, -0.3536809539, 0.0316037131, 0.4515987896, 0.9010027621, 1.3878617254, 1.9088031701, 2.4665273673, 3.0633010646, 3.6876357509, 4.3292182700, 4.9676802537, 5.5801089394, 6.1335392843, 6.5963745084, 6.9328936764, 7.1154855962, 7.1246195375, 6.9396684053, 6.5645101181, 6.0046858835, 5.2720772683, 4.3912865024, 3.3914862612, 2.3010794429, 1.1506466542, -0.0217526315, -1.1861910941, -2.3055328114, -3.3475543972, -4.2807156478, -5.0755742942, -5.7091373124, -6.1656498871, -6.4314734677, -6.5086225188, -6.4029808928, -6.1322761302, -5.7155842399, -5.1831107133, -4.5664897836, -3.8932616108, -3.1955083053, -2.4987421043, -1.8216939838, -1.1836023642, -0.5974357830, -0.0678460894, 0.4026162090, 0.8181874508, 1.1808343172, 1.5011477957, 1.7907116704, 2.0634847062, 2.3440225449, 2.6646290395, 3.0790509053, 3.6558932386, 4.4878621520, 5.6482488497]
# V1z = [3.1870074276, 2.2760237548, 1.5245170925, 0.9283769424, 0.4819600105, 0.1750562426, -0.0217722824, -0.1128347156, -0.1426070602, -0.1063781444, -0.0283509615, 0.0710321544, 0.1848522466, 0.3043940211, 0.4241210597, 0.5330613513, 0.6267014040, 0.7148312913, 0.7974297675, 0.8564132682, 0.9105517095, 0.9490980727, 0.9807646727, 1.0098203376, 1.0280471774, 1.0390154208, 1.0483889177, 1.0568166098, 1.0571336744, 1.0606348544, 1.0557108343, 1.0533362767, 1.0491301940, 1.0360955407, 1.0207349109, 1.0072672689, 0.9840704064, 0.9617231778, 0.9285378304, 0.9004992710, 0.8680153947, 0.8305311515, 0.7933930386, 0.7533402966, 0.7089048583, 0.6757089306, 0.6300704789, 0.5969553385, 0.5567831445, 0.5282148296, 0.5042546446, 0.4824423970, 0.4710582595, 0.4615529589, 0.4529255074, 0.4499248731, 0.4548176274, 0.4682095391, 0.4719265909, 0.4908074692, 0.5024710926, 0.5101040448, 0.5187373069, 0.5294410917, 0.5373526356, 0.5442011968, 0.5566816001, 0.5731919782, 0.5973037739, 0.6344643492, 0.6778787757, 0.7423648490, 0.8170723603, 0.9142122123, 1.0126312904, 1.1200776065, 1.2172993066, 1.2950790441, 1.3423741690, 1.3374931650, 1.2666037152, 1.1053318849, 0.8229646768, 0.4163146660, -0.1534318660, -0.8967277231, -1.8310470174, -2.9719339492, -4.3330644826, -5.9000816829, -7.6664670614, -9.6080548452, -11.6827724400, -13.8397639931, -16.0031014190, -18.0841577733, -19.9874556164, -21.6103629329, -22.8598305068, -23.6454169478, -23.9295474763, -23.6887943722, -22.9333608078, -21.7310509194, -20.1511938187, -18.2881746403, -16.2405878844, -14.1097954947, -11.9851838022, -9.9320606153, -8.0103672446, -6.2585204310, -4.6986106568, -3.3415579901, -2.1997141084, -1.2515611717, -0.4930768340, 0.1010114528, 0.5420514077, 0.8555977267, 1.0592916376, 1.1820195000, 1.2374677057, 1.2374081877, 1.1947023092, 1.1316541356, 1.0543115223, 0.9741061097, 0.8859538421, 0.8097622004, 0.7291816520, 0.6691906902, 0.6200359325, 0.5798871172, 0.5530782009, 0.5333667880, 0.5265733784, 0.5279092348, 0.5287804223, 0.5385582940, 0.5503264355, 0.5616601821, 0.5691193739, 0.5688268378, 0.5662910160, 0.5461864391, 0.5210433757, 0.4765168064, 0.4151294280, 0.3371007517, 0.2457107147, 0.1312762948, -0.0029687691, -0.1463482764, -0.2969746041, -0.4413842104, -0.5755051297, -0.6820120102, -0.7440820580, -0.7380497168, -0.6412569306, -0.4256414866, -0.0647069700, 0.4745143773, 1.2210743084, 2.1990039986, 3.4378136008, 4.9458051203, 6.7322446061, 8.7784174622, 11.0583660127, 13.5198234729, 16.0832331883, 18.6535854616, 21.1000567980, 23.2900906252, 25.0895046182, 26.3623091830, 27.0223610973, 27.0184938269, 26.3495833006, 25.0791667161, 23.3051307965, 21.1487922836, 18.7575879988, 16.2538143321, 13.7677391208, 11.3871618287, 9.1833845809, 7.2051778119, 5.4876689445, 4.0219173097, 2.8109443104, 1.8474143543, 1.0981042349, 0.5462314829, 0.1546296075, -0.1004452715, -0.2474881207, -0.3062536569, -0.3015012228, -0.2559812047, -0.1754982920, -0.0840909577, 0.0142267697, 0.1117985928, 0.1999403981, 0.2854675047, 0.3634591244, 0.4226783901, 0.4769007531, 0.5214122407, 0.5541678287, 0.5831613747, 0.6078456198, 0.6272108829, 0.6450838164, 0.6631219666, 0.6785862885, 0.6874098899, 0.6983353410, 0.7067679934, 0.7032713579, 0.6969634296, 0.6793267166, 0.6477066194, 0.6022108715, 0.5388132450, 0.4495588408, 0.3426055061, 0.2131461756, 0.0627503913, -0.1076845298, -0.2926323906, -0.4782687530, -0.6549448804, -0.8069785073, -0.9108495146, -0.9458290541, -0.8796022188, -0.6804789434, -0.3094226534, 0.2723424040, 1.0928637500, 2.1834221483, 3.5826995042, 5.2935910106, 7.3165203829, 9.6338350081, 12.2014935013, 14.9441285415, 17.7610319370, 20.5155498418, 23.0651784382, 25.2309822631, 26.8683329578, 27.8514864171, 28.1001586247, 27.6044487954, 26.4069928895, 24.6201989662, 22.3804726751, 19.8585052068, 17.1972097185, 14.5404757013, 11.9960311113, 9.6473143087, 7.5532846532, 5.7317366132, 4.1997060923, 2.9421361191, 1.9501828775, 1.1867165601, 0.6277799249, 0.2380966919, -0.0064397966, -0.1452897685, -0.1962462267, -0.1848277714, -0.1361671624, -0.0624719648, 0.0297423362, 0.1214051413, 0.2148793510, 0.2948667831, 0.3668244518, 0.4242475599, 0.4759419573, 0.5140932085, 0.5397395380, 0.5530933636, 0.5590608440, 0.5596942148, 0.5532313969, 0.5436539449, 0.5244731166, 0.5098630022, 0.4809469966, 0.4605275784, 0.4362241577, 0.4126720630, 0.3900785756, 0.3715263418, 0.3638730882, 0.3582007121, 0.3669746246, 0.3902114665, 0.4318613826, 0.4911047923, 0.5719780896, 0.6772601073, 0.8057256154, 0.9590140383, 1.1258074220, 1.3043072909, 1.4787096999, 1.6356671249, 1.7450225994, 1.7851913629, 1.7234475793, 1.5144175759, 1.1216893613, 0.4948448075, -0.4069298822, -1.6331530290, -3.2089335723, -5.1601127523, -7.4872962351, -10.1584835316, -13.1153868999, -16.2563533120, -19.4365608843, -22.4758495589, -25.1620563661, -27.2844170861, -28.6596705643, -29.1561841801, -28.7294212530, -27.4235462672, -25.3624608144, -22.7396056367, -19.7558298728, -16.6319919645, -13.5362858340, -10.6189762640, -7.9742554509, -5.6656377443, -3.7229985359, -2.1415556812, -0.8994200146, 0.0288355631, 0.6910688027, 1.1256330320, 1.3755938622, 1.4870570287, 1.4991496078, 1.4378427517, 1.3300620631, 1.2005134906, 1.0623141172, 0.9284048630, 0.8092555039, 0.7017037410, 0.6181911141, 0.5503666694, 0.5012939483, 0.4681723307, 0.4477198158, 0.4410476793, 0.4411675840, 0.4496795367, 0.4592889649, 0.4676134151, 0.4809817044, 0.4900695488, 0.5018048316, 0.5140973943, 0.5251397899, 0.5309367056, 0.5410020289, 0.5502821550, 0.5550590958, 0.5604285457, 0.5649003838, 0.5622568362, 0.5511804843, 0.5358689118, 0.5089924971, 0.4680373418, 0.4154305773, 0.3485611053, 0.2613275828, 0.1622434670, 0.0570430333, -0.0447121979, -0.1334973276, -0.1926143692, -0.2007169328, -0.1262982074, 0.0556691736, 0.3846700286, 0.9022945992, 1.6438714698, 2.6491813887, 3.9526299820, 5.5820364527, 7.5515807709, 9.8523526599, 12.4490937882, 15.2778700539, 18.2390896334, 21.1873182219, 23.9521806559, 26.3389808550, 28.1579415829, 29.2385678135, 29.4784589036, 28.8440254972, 27.3880781570, 25.2401655223, 22.5802900432, 19.6056156220, 16.5133281785, 13.4771151965, 10.6297144956, 8.0651157586, 5.8461182840, 3.9950173822, 2.5116342600, 1.3718086992, 0.5427267799, -0.0270505785, -0.3783172066, -0.5644811633, -0.6284939409, -0.6053041280, -0.5222157370, -0.4061076208, -0.2653360379, -0.1112895279, 0.0523807375, 0.2196710064, 0.3937496338, 0.5656616948, 0.7337174295, 0.8928007242, 1.0399490406, 1.1712935959, 1.2816465952, 1.3692512595, 1.4346865452, 1.4773994581, 1.5001962389, 1.5043766615, 1.4903822359, 1.4627591046, 1.4178860956, 1.3599141047, 1.2885186422, 1.2061462656, 1.1137346244, 1.0212786832, 0.9283897366, 0.8371677000, 0.7544576041, 0.6770066685, 0.6064903107, 0.5395206994, 0.4743040127, 0.4061534132, 0.3348929471, 0.2612606049, 0.1821695143, 0.1026212893, 0.0265769701, -0.0483597541, -0.1141174652, -0.1684512230, -0.2085961979, -0.2351678060, -0.2474102899, -0.2417129994, -0.2202571580, -0.1829626104, -0.1289331980, -0.0646364073, 0.0095309850, 0.0849583637, 0.1540513594, 0.2034662865, 0.2163268850, 0.1788967432, 0.0672063743, -0.1487877449, -0.4994712719, -1.0342699749, -1.8192684194, -2.9535566543, -4.5987342495, -7.0052649568]

# ─────────────────────────────────────
# 4. GRAPE optimization (finite-diff grad)
# ─────────────────────────────────────
history=[]
for ep in range(n_epochs+1):
    # learning-rate schedule
    lr = 1e-8 if ep<=500 else 1e-6

    if ep>0:
        f0_base,_ = f0_and_trace(V1x,V1y,V1z,
                                 theta_rot,tau,N_slices,
                                 B0,omega0,h_FD,J,projs)
        grads=[]
        eps=1e-5
        for arr in (V1x,V1y,V1z):
            g=np.zeros_like(arr)
            for i in range(N_slices):
                arr[i]+=eps
                f_pert,_=f0_and_trace(V1x,V1y,V1z,
                                      theta_rot,tau,N_slices,
                                      B0,omega0,h_FD,J,projs)
                g[i]=(f_pert-f0_base)/eps
                arr[i]-=eps
            grads.append(g)
        V1x+=lr*grads[0]; V1y+=lr*grads[1]; V1z+=lr*grads[2]

    f0_val,tr_inv=f0_and_trace(V1x,V1y,V1z,
                               theta_rot,tau,N_slices,
                               B0,omega0,h_FD,J,projs)
    history.append(tr_inv)
    if ep%10==0:
        print(f"Ep {ep:04d}: f0={f0_val:.6e}  Tr[Fcl^-1]={tr_inv:.10e}")

# ─────────────────────────────────────
# 5. Results
# ─────────────────────────────────────
print("\n# Optimised control fields:")
print("V1x = " + "[" + ", ".join(f"{v:.10f}" for v in V1x) + "]")
print("V1y = " + "[" + ", ".join(f"{v:.10f}" for v in V1y) + "]")
print("V1z = " + "[" + ", ".join(f"{v:.10f}" for v in V1z) + "]")

plt.plot(history); plt.xlabel("epoch"); plt.ylabel("Tr[F_cl^-1]")
plt.title("Training progress (B, ω)"); plt.grid(); plt.show()